<a href="https://colab.research.google.com/github/edwin-19/ASR_Exploration/blob/main/WhisperFast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jiwer
!pip install insanely-fast-whisper
!pip install datasets

In [2]:
!pip install flash-attn

In [3]:
from datasets import load_dataset
import torch
from transformers import pipeline
import jiwer

In [4]:
# Download sample wavs
dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

pipe = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-large-v3",
    torch_dtype=torch.float16,
    # model_kwargs={"use_flash_attention_2": True},
    device=device
)

pipe.model = pipe.model.to_bettertransformer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [8]:
%%time
outputs = pipe(
    dataset['file'],
    chunk_length_s=30,
    batch_size=24,
    return_timestamps=True
)

CPU times: user 26.8 s, sys: 6.73 s, total: 33.6 s
Wall time: 37.8 s


In [9]:
pred_text = [out['text'].lower() for out in outputs]
gt_text = [data['text'].lower() for data in dataset]
wer = jiwer.wer(gt_text, pred_text)
cer = jiwer.cer(gt_text, pred_text)

print(wer, cer)

0.15478260869565216 0.0443768023069529


In [10]:
from tqdm import tqdm
for gt, pred in tqdm(zip(gt_text, pred_text)):
  print('\nGT: {}\nPred: {}'.format(gt, pred))
  break

0it [00:00, ?it/s]


GT: mister quilter is the apostle of the middle classes and we are glad to welcome his gospel
Pred:  mr. quilter is the apostle of the middle classes, and we are glad to welcome his gospel.
